In [1]:
import json
import pickle as pk
import pandas as pd
from collections import defaultdict
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import cohen_kappa_score
import copy
import os

### Calculate the kappa between two annotators

In [5]:
with open('./data/docred/dev.json', 'r') as fh:
    dataset = json.load(fh)
with open('./data/docred/rel2id.json', 'r') as fh:
    rel2id = json.load(fh)
id2rel = {}
for rel, idx in rel2id.items():
    id2rel[idx] = rel

In [21]:
file_list = [249, 866, 434, 672, 184, 938, 212, 140, 546, 347, 633, 556, 205, 340, 533, 513, 130, 5, 777, 650, 20, 414, 543, 419, 813, 561, 52, 310, 698, 314, 61, 961, 198, 714, 974, 746, 753, 881, 377, 564, 270, 835, 371, 471, 692, 560, 905, 411, 38, 721, 297, 559, 803, 649, 955, 409, 380, 976, 815, 500, 480, 660, 438, 454, 718, 306, 996, 154, 620, 772, 498, 155, 787, 332, 770, 767, 196, 981, 115, 722, 609, 289, 231, 959, 124, 17, 222, 653, 214, 181, 940, 426, 685, 244, 807, 496]

name1 = 'annotator1'
name2 = 'annotator2'

same, tot, hqz, yey = 0, 0, 0, 0
pred1, pred2 = [], []

for file in file_list:
    data = dataset[file-1]
    
    infile1 = './data/annotation/{}/{}.pkl'.format(name1, file - 1)
    infile2 = './data/annotation/{}/{}.pkl'.format(name2, file - 1)
    
    data1 =  pk.load( open( infile1, "rb" ) )
    data2 = pk.load(open(infile2, 'rb'))
    
    data['pred_labels'] = []

    pred_labels1 = dict()
    pred_labels2 = dict()

    for key in data1:
        val1, val2 = data1[key], data2[key]
        for r_idx in range(97):
            rel = id2rel[r_idx]
            r1, r2 = val1[r_idx], val2[r_idx]
            if r1 in [2,3]:
                pred_labels1[(key[0], key[1], rel)] = ""
            if r2 in [2,3]:
                pred_labels2[(key[0], key[1], rel)] = ""
    union = set(pred_labels1.keys()) | set(pred_labels2.keys())
    non_nan = set()
    for key in union:
        non_nan.add((key[0], key[1]))
    nan_num = len(set(data1.keys()) - non_nan)
    for key in union:

        rel = key[2]
        if key in pred_labels1 and key in pred_labels2:
            data['pred_labels'].append({'h': key[0], 't': key[1], 'r': rel, 'evidence': [],})
            pred1.append(rel2id[rel])
            pred2.append(rel2id[rel])
        elif key in pred_labels1 and key not in pred_labels2:
            data['pred_labels'].append({'h': key[0], 't': key[1], 'r': rel, 'evidence': [],})
            pred1.append(rel2id[rel])
            pred2.append(0)
        else:
            data['pred_labels'].append({'h': key[0], 't': key[1], 'r': rel, 'evidence': [],})
            pred1.append(0)
            pred2.append(rel2id[rel])
    pred1 += [0] * nan_num
    pred2 += [0] * nan_num

same = np.sum(np.equal(pred1, pred2))
tot = len(pred1)
pred_num1 = len(np.nonzero(pred1)[0])
pred_num2 = len(np.nonzero(pred2)[0])
kappa = cohen_kappa_score(pred1, pred2)

print ("Doc Num:", len(file_list), "Inter:", same, "Union:", tot, "Annotator1 Label:", pred_num1, "Annotator2 Label:", pred_num2)
print ("Jaccard:", same / tot, "Kappa:", kappa)

Doc Num: 96 Inter: 37242 Union: 38991 Annotator1 Label: 2888 Annotator2 Label: 2621
Jaccard: 0.9551434946526122 Kappa: 0.6703530015065771


## Make sub dataset

Make Scratch、revise Dataset

In [2]:
with open('./data/annotation/reverse_rules.json', 'r') as fh:
    reverse_rules = json.load(fh)

In [3]:
for file in os.listdir('./data/annotation/discussed'):
    try:
        no = int(file.split('.')[0])
        if no + 1 not in file_list:
            os.remove('./data/annotation/discussed/{}'.format(file))
    except:
        os.remove('./data/annotation/discussed/{}'.format(file))

In [7]:
file_list = [249, 866, 434, 672, 184, 938, 212, 140, 546, 347, 633, 556, 205, 340, 533, 513, 130, 
             5, 777, 650, 20, 414, 543, 419, 813, 561, 52, 310, 698, 314, 61, 961, 198, 714, 974, 
             746, 753, 881, 377, 564, 270, 835, 371, 471, 692, 560, 905, 411, 38, 721, 297, 559, 
             803, 649, 955, 409, 380, 976, 815, 500, 480, 660, 438, 454, 718, 306, 996, 154, 620, 
             772, 498, 155, 787, 332, 770, 767, 196, 981, 115, 722, 609, 289, 231, 959, 124, 17, 
             222, 653, 214, 181, 940, 426, 685, 244, 807, 496,]

name1 = 'discussed'
name2 = 'docred'

same, tot, hqz, yey = 0, 0, 0, 0
pred1, pred2 = [], []
new_dataset = []
for file in file_list:
    data = dataset[file-1]
    infile1 = './data/annotation/{}/{}.pkl'.format(name1, file - 1)
    
    data1 =  pk.load( open( infile1, "rb" ) )
    
    data['pred_labels'] = []

    pred_labels1 = dict()
    pred_labels2 = dict()

    for key in data1:
        val1 = data1[key]
        for r_idx in range(97):
            rel = id2rel[r_idx]
            r1 = val1[r_idx]
            if r1 in [2,3]:
                pred_labels1[(key[0], key[1], rel)] = ""
                if rel in reverse_rules and len(reverse_rules[rel]) == 1:
                    reverse_rel = reverse_rules[rel][0]
                    if (key[1], key[0], reverse_rel) not in pred_labels1:
                        pred_labels1[(key[1], key[0], reverse_rel)] = "PostReverse"  

    for label in data['labels']:
        pred_labels2[(label['h'], label['t'], label['r'])] = ""
                                
    union = set(pred_labels1.keys()) | set(pred_labels2.keys())
    non_nan = set()
    for key in union:
        non_nan.add((key[0], key[1]))
    nan_num = len(set(data1.keys()) - non_nan)
    for key in union:

        rel = key[2]
        if key in pred_labels1 and key in pred_labels2:
            data['pred_labels'].append({'h': key[0], 't': key[1], 'r': rel, 'evidence': [], 'pred_label': 'TP', 'notes': ""})
            pred1.append(rel2id[rel])
            pred2.append(rel2id[rel])
        elif key in pred_labels1 and key not in pred_labels2:
            data['pred_labels'].append({'h': key[0], 't': key[1], 'r': rel, 'evidence': [], 'pred_label': 'FP', 'notes': ""})
            pred1.append(rel2id[rel])
            pred2.append(0)
        else:
            data['pred_labels'].append({'h': key[0], 't': key[1], 'r': rel, 'evidence': [], 'pred_label': 'FN', 'notes': ""})
            pred1.append(0)
            pred2.append(rel2id[rel])

    new_dataset.append(data)

with open('./data/annotation/Diff_Scratch_Revise.json'.format(name1, name2), 'w') as fh:
    json.dump(new_dataset, fh, indent=2)
same = np.sum(np.equal(pred1, pred2))
tot = len(pred1)
pred_num1 = len(np.nonzero(pred1)[0])
pred_num2 = len(np.nonzero(pred2)[0])
kappa = cohen_kappa_score(pred1, pred2)

print ("Doc Num:", len(file_list), "Inter:", same, "Union:", tot, "Scratch:", pred_num1, "DocRED:", pred_num2)
print ("Jaccard:", same / tot, "Kappa:", kappa)

Doc Num: 96 Inter: 867 Union: 3308 Scratch: 2961 DocRED: 1214
Jaccard: 0.2620918984280532 Kappa: 0.19712609763651767


In [8]:
scratch_dataset = []
revise_dataset = []
for data in new_dataset:
    new_data = copy.deepcopy(data)
    data['labels'] = [label for label in data['pred_labels'] if label['pred_label'] in ['TP', 'FP']]
    del data['pred_labels']
    scratch_dataset.append(data)
    new_data['labels'] = [label for label in new_data['pred_labels'] if label['pred_label'] in ['TP', 'FN']]
    del new_data['pred_labels']
    revise_dataset.append(new_data)
with open('./data/docred/valid_scratch.json', 'w') as fh:
    json.dump(scratch_dataset, fh, indent=2)
with open('./data/docred/valid_revise.json', 'w') as fh:
    json.dump(revise_dataset, fh, indent=2)

假设docred中标注没有错误，将revise中存在的rel全部放入scratch中

In [55]:
with open('./data/docred/valid_scratch.json', 'r') as fh:
    scratch_dataset = json.load(fh)
with open('./data/docred/valid_revise.json', 'r') as fh:
    revise_dataset = json.load(fh)
for data1, data2 in zip(scratch_dataset, revise_dataset):
    label1 = [(label['h'], label['t'], label['r']) for label in data1['labels']]
    label2 = [(label['h'], label['t'], label['r']) for label in data2['labels']]

    for label in set(label2) - set(label1):
        data1['labels'].append({'h': label[0], 't': label[1], 'r': label[2]})

with open('./data/docred/valid_scratch.json', 'w') as fh:
    json.dump(scratch_dataset, fh, indent=2)

Make Recommend Dataset

In [41]:
tot_labels = 0

with open('./data/docred/init.json', 'r') as fh:
    init_dataset = json.load(fh)
with open('./data/docred/dev.json', 'r') as fh:
    dev_dataset = json.load(fh)
with open('./data/docred/dev.json', 'r') as fh:
    valid_recommend = json.load(fh)
for idx, data in enumerate(dev_dataset):
    for init_data in init_dataset:
        if init_data['title'] == data['title']:
            valid_recommend[idx] = init_data
            break
# for i in [0] + list(range(2, 1000)):
#     vertex_old2new = {}
#     for old_idx, old_nodes in enumerate(valid_recommend[i]['vertexSet']):
#         for new_idx, new_nodes in enumerate(dev_dataset[i]['vertexSet']):
#             if len(set([(t['sent_id'], t['pos'][0], t['pos'][1]) for t in old_nodes]) & set([(t['sent_id'], t['pos'][0], t['pos'][1]) for t in new_nodes])) > 0:
#                 vertex_old2new[old_idx] = new_idx
#                 break
#     labels = []
#     label_set = set()
#     for label in valid_recommend[i]['init']:
#         if label['h'] in vertex_old2new and label['t'] in vertex_old2new and vertex_old2new[label['h']] != vertex_old2new[label['t']]:
#             if (vertex_old2new[label['h']], vertex_old2new[label['t']], label['r']) not in label_set:
#                 labels.append({'h': vertex_old2new[label['h']], 't': vertex_old2new[label['t']], 'r': label['r']})
#                 label_set.add((vertex_old2new[label['h']], vertex_old2new[label['t']], label['r']))   
# 
for i in [0] + list(range(2, 1000)):
    vertex_old2new = defaultdict(list)
    for old_idx, old_nodes in enumerate(valid_recommend[i]['vertexSet']):
        for new_idx, new_nodes in enumerate(dev_dataset[i]['vertexSet']):
            if len(set([(t['sent_id'], t['pos'][0], t['pos'][1]) for t in old_nodes]) & set([(t['sent_id'], t['pos'][0], t['pos'][1]) for t in new_nodes])) > 0:
                # vertex_old2new[old_idx].append(new_idx)
                vertex_old2new[old_idx] = [new_idx]
                break
                
    labels = []
    label_set = set()
    for label in valid_recommend[i]['init']:
        if label['h'] in vertex_old2new and label['t'] in vertex_old2new:
            for h in vertex_old2new[label['h']]:
                for t in vertex_old2new[label['t']]:
                    if h == t:
                        continue
                    if (h, t, label['r']) not in label_set:
                        labels.append({'h': h, 't': t, 'r': label['r']})
                        label_set.add((h, t, label['r'])) 
                
    tot_labels += len(set([(label['h'], label['t'], label['r']) for label in labels]))
    assert len(set([(label['h'], label['t'], label['r']) for label in labels])) == len(label_set)

    dev_dataset[i]['labels'] = labels

print (tot_labels)

25170


In [42]:
with open('./data/docred/valid_revise.json', 'r') as fh:
    revise_dataset = json.load(fh)

In [43]:
file_list = [249, 866, 434, 672, 184, 938, 212, 140, 546, 347, 633, 556, 205, 340, 533, 513, 130, 
             5, 777, 650, 20, 414, 543, 419, 813, 561, 52, 310, 698, 314, 61, 961, 198, 714, 974, 
             746, 753, 881, 377, 564, 270, 835, 371, 471, 692, 560, 905, 411, 38, 721, 297, 559, 
             803, 649, 955, 409, 380, 976, 815, 500, 480, 660, 438, 454, 718, 306, 996, 154, 620, 
             772, 498, 155, 787, 332, 770, 767, 196, 981, 115, 722, 609, 289, 231, 959, 124, 17, 
             222, 653, 214, 181, 940, 426, 685, 244, 807, 496,]
valid_recommend = [dev_dataset[idx-1] for idx in file_list]
for revise_data, recommend_data in zip(revise_dataset, valid_recommend):
    revise_labels = [(label['h'], label['t'], label['r']) for label in revise_data['labels']]
    recommend_labels = [(label['h'], label['t'], label['r']) for label in recommend_data['labels']]
    recommend_labels = set(recommend_labels) & set(revise_labels) # 只考虑在revise阶段保留的triple
    recommend_data['labels'] = [{'h': l[0], 't': l[1], 'r': l[2]} for l in recommend_labels]
with open('./data/docred/valid_recommend.json', 'w') as fh:
    json.dump(valid_recommend, fh, indent=2)

In [44]:
l_recommend = [len(data['labels']) for data in valid_recommend]
l_revise = [len(data['labels']) for data in revise_dataset]

In [25]:
l_recommend[:10]

[4, 6, 16, 12, 3, 9, 8, 18, 8, 13]

In [26]:
l_revise[:10]

[4, 6, 18, 13, 3, 9, 10, 18, 8, 13]

## Compare Dataset

In [56]:
valid_dataset = {}
with open('./data/docred/valid_scratch.json', 'r') as fh:
    valid_dataset['scratch'] = json.load(fh)
with open('./data/docred/valid_recommend.json', 'r') as fh:
    valid_dataset['recommend'] = json.load(fh)
with open('./data/docred/valid_revise.json', 'r') as fh:
    valid_dataset['revise'] = json.load(fh)

In [57]:
valid_dataset['scratch_recommend'] = []
valid_dataset['revise_recommend'] = []
valid_dataset['scratch_revise'] = []

for scratch_data, revise_data, recommend_data in zip(valid_dataset['scratch'], valid_dataset['revise'], valid_dataset['recommend']):
    scratch_labels = set([(l['h'], l['t'], l['r']) for l in scratch_data['labels']])
    revise_labels = set([(l['h'], l['t'], l['r']) for l in revise_data['labels']])
    recommend_labels = set([(l['h'], l['t'], l['r']) for l in recommend_data['labels']])

    data = copy.deepcopy(revise_data)
    data['labels'] = [{'h': l[0], 't': l[1], 'r': l[2]} for l in scratch_labels - recommend_labels]
    valid_dataset['scratch_recommend'].append(data)

    data = copy.deepcopy(revise_data)
    data['labels'] = [{'h': l[0], 't': l[1], 'r': l[2]} for l in scratch_labels - revise_labels]
    valid_dataset['scratch_revise'].append(data)

    data = copy.deepcopy(revise_data)
    data['labels'] = [{'h': l[0], 't': l[1], 'r': l[2]} for l in revise_labels - recommend_labels]
    valid_dataset['revise_recommend'].append(data)

Popular vs Unpopular relations

In [58]:
with open('./data/docred/rel2id.json', 'r') as fh:
    rel2id = json.load(fh)
with open('./data/wikidata/Popular Relations.txt' ,'r') as fh:
    lines = fh.readlines()
popular_relations = [line.strip().split('\t')[0].split(' ')[-1][1:-1] for line in lines]
popular_relations = set(popular_relations) & set(list(rel2id))
unpopular_relations = [rel for rel in rel2id if rel not in popular_relations and rel != 'Na']

In [74]:
dt = {
    '# Instance': [],
    '# Pop Rel': [],
    '# Unpop Rel': [],
    '$popularity_{max}$': [],
    '$popularity_{min}$': []
}

In [75]:
for data_name in ['recommend', 'revise', 'scratch', 'revise_recommend', 'scratch_recommend', 'scratch_revise']:
    dataset = valid_dataset[data_name]
    ins_with_rel = defaultdict(list)
    for data in dataset:
        for label in data['labels']:
            ins_with_rel[label['r']].append((data['title'], label['h'], label['t'], rel2id[label['r']]))
    pop_rel_instances = []
    unpop_rel_instances = []
    for rel in popular_relations:
        pop_rel_instances += ins_with_rel[rel]
    for rel in unpopular_relations:
        unpop_rel_instances += ins_with_rel[rel]

    pop_rel_num = len(set(pop_rel_instances))
    unpop_rel_num = len(set(unpop_rel_instances))

    print ("Data: {}, #Instances: {}, #Popular Rel: {}({:.1f}%), #Unpopular Rel: {}({:.1f}%)".format(
        data_name,
        pop_rel_num + unpop_rel_num,
        pop_rel_num, pop_rel_num / (pop_rel_num + unpop_rel_num) * 100,
        unpop_rel_num, unpop_rel_num / (pop_rel_num + unpop_rel_num) * 100,
    ))
    dt['# Instance'].append(pop_rel_num + unpop_rel_num)
    dt['# Pop Rel'].append("{} ({}%)".format(pop_rel_num, format(100 * pop_rel_num / (pop_rel_num + unpop_rel_num), ".1f") ))
    dt['# Unpop Rel'].append("{} ({}%)".format(unpop_rel_num, format(100 * unpop_rel_num / (pop_rel_num + unpop_rel_num), ".1f")))


Data: recommend, #Instances: 1167, #Popular Rel: 659(56.5%), #Unpopular Rel: 508(43.5%)
Data: revise, #Instances: 1214, #Popular Rel: 676(55.7%), #Unpopular Rel: 538(44.3%)
Data: scratch, #Instances: 3308, #Popular Rel: 1615(48.8%), #Unpopular Rel: 1693(51.2%)
Data: revise_recommend, #Instances: 47, #Popular Rel: 17(36.2%), #Unpopular Rel: 30(63.8%)
Data: scratch_recommend, #Instances: 2141, #Popular Rel: 956(44.7%), #Unpopular Rel: 1185(55.3%)
Data: scratch_revise, #Instances: 2094, #Popular Rel: 939(44.8%), #Unpopular Rel: 1155(55.2%)


Entity Popularity

In [60]:
with open('./data/wikidata/final_id2popular.json', 'r') as fh:
    id2popular = json.load(fh)
with open('./data/wikidata/final_wiki_ids.json', 'r') as fh:
    wiki_ids = json.load(fh)

with open('./data/docred/dev.json', 'r') as fh:
    dev_dataset = json.load(fh)
doc_titles = [data['title'] for data in dev_dataset]
title2id = {}
for fid, title in enumerate(doc_titles):
    title2id[title] = fid

In [50]:
# with open('/home/huangqz/repos/binary_document_re/GAIN-dsn/code/wiki/ent2feature.json', 'r') as fh:
#     ent2feature = json.load(fh)

# with open('/home/huangqz/repos/binary_document_re/GAIN-dsn/code/wiki/real_ent2feature.json', 'r') as fh:
#     real_ent2feature = json.load(fh)


# id2popular = {}
# for qid in ent2feature:
#     id2popular[qid] = sum(len(rel) for rel in ent2feature[qid]['rel'])
# for qid in real_ent2feature:
#     id2popular[qid] = sum(len(rel) for rel in real_ent2feature[qid]['rel'])

# with open('./data/wikidata/final_id2popular.json', 'w') as fh:
#     json.dump(id2popular, fh, indent=2)

In [51]:
# with open('/home/huangqz/repos/binary_document_re/GAIN-dsn/code/wiki/real_wiki_ids.json', 'r') as fh:
#     wiki_ids = json.load(fh)

# with open('/home/huangqz/repos/binary_document_re/GAIN-dsn/code/wiki/wiki_ids.json', 'r') as fh:
#     wiki_ids_backup = json.load(fh)

# for doc_wiki_ids, doc_wiki_ids_backup in zip(wiki_ids, wiki_ids_backup):
#     for i, ent_wiki_ids in enumerate(doc_wiki_ids):
#         if len(ent_wiki_ids) == 0:
#             doc_wiki_ids[i] = doc_wiki_ids_backup[i]
# wiki_ids[1] = wiki_ids_backup[1]

# with open('./data/wikidata/final_wiki_ids.json', 'w') as fh:
#     json.dump(wiki_ids, fh, indent=2)

In [61]:
def get_popular_with_title_hts(title, h, t, merge_method='max'):
    doc_id = title2id[title]
    h_wiki_ids = wiki_ids[doc_id][int(h)]
    t_wiki_ids = wiki_ids[doc_id][int(t)]

    h_popularity = [id2popular[qid] for qid in h_wiki_ids if qid in id2popular]
    t_popularity = [id2popular[qid] for qid in t_wiki_ids if qid in id2popular]

    h_popularity = -1 if len(h_popularity) == 0 else max(h_popularity)
    t_popularity = -1 if len(t_popularity) == 0 else max(t_popularity)

    if merge_method == 'max':
        return max(h_popularity, t_popularity)
    elif merge_method == 'min':
        return min(h_popularity, t_popularity)
    elif merge_method == 'ave':
        return (h_popularity + t_popularity) / 2

In [76]:
for data_name in ['recommend', 'revise', 'scratch', 'revise_recommend', 'scratch_recommend', 'scratch_revise']:
    dataset = valid_dataset[data_name]
    max_popularity = [get_popular_with_title_hts(data['title'], label['h'], label['t'], 'max') for data in dataset for label in data['labels']]
    min_popularity = [get_popular_with_title_hts(data['title'], label['h'], label['t'], 'min') for data in dataset for label in data['labels']]
    print ("Data: {}, #Instances: {}, #Max Popularity: {:.1f}, #Min Popularity: {:.1f}".format(
        data_name, len(max_popularity), np.mean(max_popularity), np.mean(min_popularity)))
    dt['$popularity_{max}$'].append(format(np.mean(max_popularity), ".1f"))
    dt['$popularity_{min}$'].append(format(np.mean(min_popularity), ".1f"))



Data: recommend, #Instances: 1167, #Max Popularity: 294.4, #Min Popularity: 85.2
Data: revise, #Instances: 1214, #Max Popularity: 291.5, #Min Popularity: 84.4
Data: scratch, #Instances: 3308, #Max Popularity: 266.3, #Min Popularity: 67.4
Data: revise_recommend, #Instances: 47, #Max Popularity: 221.3, #Min Popularity: 66.0
Data: scratch_recommend, #Instances: 2141, #Max Popularity: 251.0, #Min Popularity: 57.7
Data: scratch_revise, #Instances: 2094, #Max Popularity: 251.7, #Min Popularity: 57.5


In [78]:
df = pd.DataFrame(dt)

In [80]:
index = ["$mathbf{D_{Recommend}}$", "$mathbf{D_{Revise}}$", "$mathbf{D_{Scratch}}$", "$mathbf{D_{Revise} - D_{Recommend}}$", "$mathbf{D_{Scratch} - D_{Recommend}}$", "$mathbf{D_{Scratch} - D_{Revise}}$"]
df.index = index

In [81]:
print(df.to_latex().replace("\\$", "$").replace("\\_", "_").replace("\\{", "{").replace("\\}", "}").replace("mathbf", "\mathbf"))

\begin{tabular}{lrllll}
\toprule
{} &  \# Instance &     \# Pop Rel &   \# Unpop Rel & $popularity_{max}$ & $popularity_{min}$ \\
\midrule
$\mathbf{D_{Recommend}}$               &        1167 &   659 (56.5\%) &   508 (43.5\%) &              294.4 &               85.2 \\
$\mathbf{D_{Revise}}$                  &        1214 &   676 (55.7\%) &   538 (44.3\%) &              291.5 &               84.4 \\
$\mathbf{D_{Scratch}}$                 &        3308 &  1615 (48.8\%) &  1693 (51.2\%) &              266.3 &               67.4 \\
$\mathbf{D_{Revise} - D_{Recommend}}$  &          47 &    17 (36.2\%) &    30 (63.8\%) &              221.3 &               66.0 \\
$\mathbf{D_{Scratch} - D_{Recommend}}$ &        2141 &   956 (44.7\%) &  1185 (55.3\%) &              251.0 &               57.7 \\
$\mathbf{D_{Scratch} - D_{Revise}}$    &        2094 &   939 (44.8\%) &  1155 (55.2\%) &              251.7 &               57.5 \\
\bottomrule
\end{tabular}

